In [1]:
#!/usr/bin/env python

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from utils import dlc_practical_prologue as prologue
from utils.helpers import compute_nb_errors, train_model, update_target_type

from models.proj1.weightSharing import WeightSharing

import torch

from torch import optim
from torch.autograd import Variable

import matplotlib
import cv2

In [2]:
# load the data
train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [3]:
def standard_wrapper(*data):
    return [Variable(t) for t in data]

In [4]:
def custom_wrapper(*data):
    return None

In [5]:
def wrap_data(*data, type_='standard'):
    switch = {
        'standard': standard_wrapper,
        'Net3': custom_wrapper
    }
    
    return switch.get(type_, None)(*data)    

In [101]:
def summary(model, optimizer, lr, training_time=None, NB_EPOCHS=None, MINI_BATCH_SIZE=None):
    print('model: {:>5}, criterion: {:>10}, optimizer: {:>10}, learning rate: {:6}, num epochs: {:3}, '
    'mini batch size: {:3}, training time: {:5.2f}, train error: {:5.2f}%, test error: {:5.2f}%'.format(
        model.__class__.__name__,
        model.criterion.__class__.__name__,
        opt.__class__.__name__,
        lr,
        NB_EPOCHS,
        MINI_BATCH_SIZE,
        training_time,
        compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
        compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100
        )
    )

In [102]:
# Compute mean and std for data
mean, std = train_input.mean(), train_input.std()

# Standardize data
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

# Wrap data in a Variable object
train_input, train_target, train_classes, test_input, test_target = \
    wrap_data(train_input, train_target, train_classes, test_input, test_target, type_='standard')


train_input_Net3 = train_input.view(-1, 1, 14, 14)
train_target_Net3 = train_classes.flatten()

# test different configurations
NB_EPOCHS = 25
MINI_BATCH_SIZE = 100
models = [WeightSharing]
opts = [torch.optim.SGD, torch.optim.Adam]
lrs = [1e-1, 1e-2, 1e-3]


for m in models:
    model = m()
    train_target, test_target = update_target_type(model, train_target, test_target)
    for opt in opts[:1]:
        for lr in lrs[:1]:      
            tt = train_model(model, opt(model.parameters(), lr=lr), NB_EPOCHS, \
                    train_input, train_target, MINI_BATCH_SIZE)
            
            summary(model, opt, lr, training_time=tt, NB_EPOCHS=NB_EPOCHS, MINI_BATCH_SIZE=MINI_BATCH_SIZE)                   

model: WeightSharing, criterion: BCEWithLogitsLoss, optimizer:       type, learning rate:    0.1, num epochs:  25, mini batch size: 100, training time:  9.83, train error:  2.40%, test error: 13.70%


In [100]:
optim = torch.optim.SGD(WeightSharing().parameters(), lr=0.001)

optim.param_groups[0]['lr']

0.001

In [8]:
import time
from datetime import datetime
import visdom

In [69]:
class Visualization():
    def __init__(self, env_name=None):
        if env_name is None:
            env_name = str(datetime.now().strftime("%d-%m %Hh%M"))
        self.env_name = env_name
        self.vis = visdom.Visdom(env=self.env_name)
        self.loss_win = None
        
    def plot_loss(self, loss, epoch, opt_name):
        self.loss_win = self.vis.line(
            [loss],
            [epoch],
            win=self.loss_win,
            update='append' if self.loss_win else None,
            opts=dict(
                xlabel='epoch',
                ylabel=opt_name,
                title='Loss (mean per 10 steps)',
            )
        )

In [94]:
def train_model(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):
    n_samples = train_input.size(0)
    
    vis = Visualization()
    losses = []
    start = time.time()
    for e in range(0,nb_epochs):
        running_loss = 0.0
        for b in range(0, n_samples, mini_batch_size):
            # zero the parameter gradients
            model.zero_grad()
            # Forward pass
            output = model(train_input.narrow(0, b, mini_batch_size))
            # Get corresponding target values
            target = train_target.narrow(0, b, mini_batch_size)
            # Backward pass
            loss = model.criterion(output, target)
            loss.backward()
            # Update weights
            optimizer.step()
            # Update loss
            running_loss += loss.item()
        losses.append(running_loss)
        vis.plot_loss(running_loss, e, optimizer.__class__.__name__)
    end = time.time()

    training_time = end-start

    return training_time